To get started, click "Runtime" -> "Run all" to run the code. There will be prompts for user input after code cells 3 and 4. At the end, a CSV file will be downloaded to your computer.

In [ ]:
### IMPORTS ###
import time
import requests
import math
import pandas as pd
from google.colab import files

In [ ]:
bearer_token = "<INSERT TOKEN HERE>"

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

def connect(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [ ]:
#get user input
user_input = input("Please enter target username (case sensitive):\t").strip()
usernames = "usernames={}".format(user_input)
uname = usernames.split("=", 1)[1]

user_fields = "user.fields=description,created_at,public_metrics"
# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)

json_response = connect(url)
num_followers = json_response["data"][0]["public_metrics"]["followers_count"]
target_id = json_response["data"][0]["id"]
print("{} has {} followers. The twitter API only allows 15 calls in a 15 minute period. Each call retrieves 1000 usernames (max) so it will take {} minutes to fetch all follower data.".format(uname, num_followers, math.ceil(num_followers / 1000)))
# print(json.dumps(json_response, indent=4, sort_keys=True))

Please enter target username (case sensitive):
	ILYYWNFT
ILYYWNFT has 80542 followers. The twitter API only allows 15 calls in a 15 minute period. Each call retrieves 1000 usernames (max) so it will take 81 minutes to fetch all follower data.


In [ ]:
num_desired = int(input("Please enter the number of desired usernames in multiples of 1000. If you would like all usernames, just hit ENTER\n\t"))
call_counter = 0
call_count = 0
if num_desired == "":
    call_count = math.ceil(num_followers / 1000)
else:
    call_count = math.ceil(num_desired / 1000)


url = "https://api.twitter.com/2/users/{}/followers?max_results=1000".format(target_id)
call_count -= 1
call_counter += 1
json_response = connect(url)
# print(json.dumps(json_response, indent=4, sort_keys=True))
response_usernames = []
for i in json_response["data"]:
    response_usernames.append(i["username"])

while json_response["meta"]["next_token"] and call_count != 0:
    if call_counter >= 14:
        print("Waiting 15 minutes...")
        time.sleep(15 * 60 + 2)
    url = "https://api.twitter.com/2/users/{}/followers?max_results=1000&pagination_token={}".format(target_id, json_response["meta"]["next_token"])
    call_counter += 1
    call_count -= 1
    json_response = connect(url)
    for i in json_response["data"]:
        response_usernames.append(i["username"])

Please enter the number of desired usernames in multiples of 1000. If you would like all usernames, just hit ENTER
	2000


In [ ]:
df = pd.DataFrame(data={"usernames": response_usernames})

fn = "{}_usernames.csv".format(uname)
df.to_csv(fn, encoding="utf-8", index=False)

files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>